# Add targets for camera calibration

In [ ]:
import os
from glob import glob
import pandas as pd
import geopandas as gpd
import numpy as np
from tqdm import tqdm
from shapely.geometry import Point
import matplotlib.pyplot as plt
import rioxarray as rxr
import xarray as xr

inputs_folder = '/Users/rdcrlrka/Research/Soo_locks/inputs'
cam_positions_file = os.path.join(inputs_folder, 'cams.txt')
ortho_files = sorted(glob(os.path.join(inputs_folder, '..', 'outputs', 'soo_locks_photogrammetry_20251001171000', 'orthoimages', '*.tiff')))
refdem_file = os.path.join(inputs_folder, 'lidar_DSM_filled_cropped.tif')
refl_file = os.path.join(inputs_folder, '20251001_Soo_Model_1cm_Intensity_UTM19N-fake.tif')

output_folder = os.path.join(inputs_folder, '..', 'add_targets')
os.makedirs(output_folder, exist_ok=True)

# Load camera positions
cams = pd.read_csv(cam_positions_file, sep=' ', header=0)
cams['channel'] = [x.split('_')[1] for x in cams['img_name']]

# imoprt utility functions
import model_camera_coverage_utils as utils

# Load current target locations
targets_file = os.path.join(inputs_folder, '..', 'add_targets', 'current_targets.gpkg')
targets = gpd.read_file(targets_file)
# grab x and y coordinates
targets['X'] = [x.coords.xy[0][0] for x in targets['geometry']]
targets['Y'] = [x.coords.xy[1][0] for x in targets['geometry']]
# sort by y-value
targets = targets.sort_values(by='Y').reset_index(drop=True)

# Load lidar DSM
dem = rxr.open_rasterio(refdem_file).squeeze()
dem = xr.where(dem==-9999, np.nan, dem)
# upsample to speed up plotting
new_x = np.arange(dem.x.min().data, dem.x.max().data, step=0.05)
new_y = np.arange(dem.y.min().data, dem.y.max().data, step=0.05)
dem = dem.interp(coords={'x': new_x, 'y': new_y}, method='linear')

## Calculate current image bounds

In [ ]:
print('Creating polygon of model space')
model_space_gdf = utils.calculate_image_footprint(refdem_file)

print('Calculating image footprints')
gdf_list = []
for ortho_file in tqdm(ortho_files):
    gdf = utils.calculate_image_footprint(ortho_file)
    gdf_list += [gdf]
bounds_df = pd.concat(gdf_list).reset_index(drop=True)
bounds_gdf = gpd.GeoDataFrame(bounds_df, geometry=bounds_df['geometry'], crs=gdf.crs)

print('Calculating number of targets in each image')
def count_image_targets(targets_gdf, image_bounds_gdf):
    df_list = []
    for i in range(len(image_bounds_gdf)):
        im_gdf = image_bounds_gdf.iloc[i]
        intersects = [im_gdf['geometry'].intersects(x) for x in targets_gdf['geometry']]
        num_intersecting = np.sum(np.array(intersects))
        df_list += [pd.DataFrame({
            'channel': [im_gdf['channel']],
            'num_targets': [num_intersecting]
        }, index=[i])]
    df_full = pd.concat(df_list)
    return df_full


image_targets_df = count_image_targets(targets, bounds_gdf)
image_targets_df

## Add targets

In [ ]:
new_target_pts = []

# Add some between the current ones
def add_targets_between(i, j, targets_gdf, n_points):
    # extract endpoints
    x1, y1 = targets_gdf.iloc[i][['X', 'Y']]
    x2, y2 = targets_gdf.iloc[j][['X', 'Y']]
    # parameter values between 0 and 1 (exclude endpoints)
    ts = np.linspace(0, 1, n_points + 2)[1:-1]
    points = [
        Point(x1 + (x2 - x1) * t, y1 + (y2 - y1) * t)
        for t in ts
    ]
    return points
new_target_pts += add_targets_between(0, 2, targets, n_points=2)
new_target_pts += add_targets_between(1, 3, targets, n_points=2)
# new_target_pts += add_targets_between(2, 4, targets, n_points=1)
# new_target_pts += add_targets_between(3, 5, targets, n_points=1)
# new_target_pts += add_targets_between(4, 6, targets, n_points=1)
# new_target_pts += add_targets_between(5, 7, targets, n_points=1)
new_target_pts += add_targets_between(6, 8, targets, n_points=1)
new_target_pts += add_targets_between(7, 10, targets, n_points=3)
new_target_pts += add_targets_between(8, 9, targets, n_points=1)
new_target_pts += add_targets_between(9, 11, targets, n_points=1)
new_target_pts += add_targets_between(10, 12, targets, n_points=1)
new_target_pts += add_targets_between(11, 13, targets, n_points=1)
new_target_pts += add_targets_between(12, 14, targets, n_points=1)
new_target_pts += add_targets_between(13, 15, targets, n_points=2)
new_target_pts += add_targets_between(14, 16, targets, n_points=2)
new_target_pts += add_targets_between(15, 17, targets, n_points=1)
new_target_pts += add_targets_between(16, 18, targets, n_points=1)
new_target_pts += add_targets_between(17, 19, targets, n_points=1)
new_target_pts += add_targets_between(18, 20, targets, n_points=1)
new_target_pts += add_targets_between(19, 21, targets, n_points=1)
new_target_pts += add_targets_between(20, 22, targets, n_points=1)
new_target_pts += add_targets_between(22, 23, targets, n_points=1)


# Add some along the walkway
new_target_pts += [
    Point(-4.5, 2.3),
    Point(-1.5, 3.1),
    Point(-5.3, 5.4),
    Point(-2.4, 6.5),
    Point(-6.3, 9.0),
    Point(-3.3, 10.0),
    Point(-6.8, 12.3),
    Point(-4.3, 13.0),
    Point(-6.2, 15.7),
    Point(-7.0, 18.9),
    Point(-7.9, 23.3),
    Point(-8.8, 26.7),
    Point(-9.6, 30.4),
    Point(-10.8, 34.4),
    Point(-11.8, 38.0),
    Point(-12.4, 40.6)
]

# Add points at the "northern end"
new_target_pts += [
    Point(-20.3, 52.5),
    Point(-21.4, 54.5),
    Point(-22.5, 56.5),
    Point(-23.6, 58.5),
    Point(-13.8, 58.7),
    Point(-14.0, 60.5)
]

# Create geodataframe
new_targets = gpd.GeoDataFrame(geometry=new_target_pts, crs="EPSG:32619")

# Merge with original targets
new_targets_full = pd.concat([targets, new_targets]).reset_index(drop=True)

# Calculate new number of targets per image
new_image_targets_df = count_image_targets(new_targets_full, bounds_gdf)
new_image_targets_df

# Compare to original
image_targets_merged_df = pd.merge(image_targets_df, new_image_targets_df, on='channel', suffixes=['_current', '_new'])

# Save to file
out_file = os.path.join(output_folder, 'targets_per_image.csv')
image_targets_merged_df.to_csv(out_file, index=False)
print(f'Targets per image saved to:\n{out_file}')

## Plot results

In [ ]:
plt.rcParams.update({'font.size': 12, 'font.sans-serif': 'Verdana'})
fig, ax = plt.subplots(figsize=(8,14))
# model space
model_space_gdf.plot(facecolor='None', edgecolor='k', ax=ax)

# lidar DSM
(xr.where(dem > -8, np.nan, dem)).plot(cmap='Grays', vmin=-8.2, vmax=-7.9, add_colorbar=False)

# image bounds
# bounds_gdf.plot(facecolor='b', edgecolor='b', alpha=0.5, linewidth=0.5, ax=ax)

# current targets
targets.plot(ax=ax, facecolor='k', edgecolor='w', linewidth=0.5, markersize=50)

# new targets
# for i in range(len(new_targets)):
#     ax.plot(new_targets.iloc[i]['geometry'].coords.xy[0][0], new_targets.iloc[i]['geometry'].coords.xy[1][0], '.', 
#             markersize=10, color=plt.cm.viridis(i/len(new_targets)))
new_targets.plot(ax=ax, facecolor='m', edgecolor='w', linewidth=0.5, markersize=50)

# dummy points for legend
xlim = ax.get_xlim()
ylim = ax.get_ylim()
ax.plot(-1000,0,'ok', label='Current targets')
ax.plot(-1000,0,'om', label='New targets')
ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.legend(loc='upper right')

ax.set_title(f'Add {len(new_targets)} targets')
ax.set_xlabel('meters')
ax.set_ylabel('meters')
fig.tight_layout()
plt.show()

fig_file = os.path.join(output_folder, f'add_{len(new_targets)}_targets.png')
fig.savefig(fig_file, dpi=300, bbox_inches='tight')
print(f"Figure saved to:\n{fig_file}")

In [ ]:
len(targets)